In [59]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
# import dataset
df = pd.read_csv('../data/international_matches.csv')
df.tournament.value_counts()

# # filter to just FIFA World Cup or FIFA World Cup qualification
# wc = df[(df.tournament == 'FIFA World Cup') | (df.tournament == 'FIFA World Cup qualification')]

Friendly                                8558
FIFA World Cup qualification            5528
UEFA Euro qualification                 1723
African Cup of Nations qualification    1274
AFC Asian Cup qualification              541
                                        ... 
AFF Championship qualification             2
Copa América qualification                 2
Copa Confraternidad                        1
TIFOCO Tournament                          1
CONMEBOL–UEFA Cup of Champions             1
Name: tournament, Length: 82, dtype: int64

In [61]:
# if at least 1 nan in the columns: home_team_goalkeeper_score, away_team_goalkeeper_score, home_team_mean_defense_score, home_team_mean_offense_score, home_team_mean_midfield_score, away_team_mean_defense_score, away_team_mean_offense_score, away_team_mean_midfield_score, then drop the row
wc_cleaned = df.dropna(subset=['home_team_goalkeeper_score', 'away_team_goalkeeper_score', 'home_team_mean_defense_score', 'home_team_mean_offense_score', 'home_team_mean_midfield_score', 'away_team_mean_defense_score', 'away_team_mean_offense_score', 'away_team_mean_midfield_score'])

# check wc_cleaned
wc_cleaned

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8264,2004-09-03,Spain,Scotland,Europe,Europe,3,67,0,0,1,...,No,Draw,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8
8266,2004-09-04,Austria,England,Europe,Europe,90,7,0,0,2,...,No,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,0,0,3,...,No,Win,77.0,74.0,80.5,78.7,79.0,71.8,75.7,70.2
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,0,0,1,...,No,Lose,78.0,78.0,68.8,77.0,69.2,70.5,79.7,78.5
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,0,0,2,...,No,Win,97.0,79.0,91.8,92.3,87.5,79.2,81.3,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,No,Lose,87.0,89.0,75.2,84.7,75.8,80.8,85.7,85.5
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,No,Draw,75.0,75.0,74.8,78.7,80.0,76.5,72.7,73.8
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,No,Win,76.0,83.0,74.2,77.0,78.0,70.0,72.3,73.5
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,No,Lose,77.0,65.0,73.5,73.7,75.0,76.2,74.7,68.2


In [62]:
# import fifa_team_ranking.csv
team_rankings = pd.read_csv('../data/fifa_team_ranking.csv')

# extract columns: rank, country_full, country_abrv, total_points, preivous_points, and rank_date from team_rankings
team_rankings = team_rankings[['rank', 'country_full', 'country_abrv', 'total_points', 'previous_points', 'rank_date']]
team_rankings

,rank,country_full,country_abrv,total_points,previous_points,rank_date
0,1,Germany,GER,0.0,57,1993-08-08
1,2,Italy,ITA,0.0,57,1993-08-08
2,3,Switzerland,SUI,0.0,50,1993-08-08
3,4,Sweden,SWE,0.0,55,1993-08-08
4,5,Argentina,ARG,0.0,51,1993-08-08
...,...,...,...,...,...,...
57788,206,Anguilla,AIA,0.0,0,2018-06-07
57789,206,Bahamas,BAH,0.0,0,2018-06-07
57790,206,Eritrea,ERI,0.0,0,2018-06-07
57791,206,Somalia,SOM,0.0,0,2018-06-07


## Experiment Imputing for Spain Matches

In [63]:
# check where spain played either as the home or away team
spain_games = wc_cleaned[(wc_cleaned.home_team == 'Spain') | (wc_cleaned.away_team == 'Spain')]

# extract first date of spain_games
match_date = spain_games.date.min()
match_date

'2004-09-03'

In [64]:
# check fifa rankings of spain
spain_rankings = team_rankings[team_rankings.country_full == 'Spain']

# find rank_date > match_date
spain_rankings_afterMatch = spain_rankings[spain_rankings.rank_date > match_date]
spain_rankings_afterMatch

# take the previous_points from the first row of spain_rankings_afterMatch
spain_rankings_afterMatch.previous_points.iloc[0]

788

## Expanding to all Matches

In [65]:
# replace FYR Macedonia in home_team and away_team with North Macedonia in team_rankings
team_rankings.replace('FYR Macedonia', 'North Macedonia', inplace=True)

# replace Cape Verde Islands in home_team and away_team with Cape Verde in team_rankings
team_rankings.replace('Cape Verde Islands', 'Cabo Verde', inplace=True)

In [66]:
# value_counts of home_team and away_team in wc_cleaned
wc_cleaned.away_team.value_counts()

# check which countries in wc_cleaned are not in team_rankings
wc_cleaned[~wc_cleaned.away_team.isin(team_rankings.country_full)]

# # find all home_team where home_team_continent is Africa
# wc_cleaned[wc_cleaned.home_team_continent == 'Africa']

# # find iran in wc_cleaned
# wc_cleaned[wc_cleaned.home_team == 'IR Iran']

# # find in team_rankings where country_full contains Iran
# team_rankings[team_rankings.country_full.str.contains('Malaysia')]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score


In [67]:
# find where 'Cape Verde' is in team_rankings
team_rankings[team_rankings.country_full.str.contains('Cabo Verde')]

,rank,country_full,country_abrv,total_points,previous_points,rank_date
142,143,Cabo Verde,CPV,0.00,2,1993-08-08
308,142,Cabo Verde,CPV,0.00,2,1993-09-23
475,142,Cabo Verde,CPV,0.00,2,1993-10-22
642,142,Cabo Verde,CPV,0.00,2,1993-11-19
815,147,Cabo Verde,CPV,0.00,2,1993-12-23
...,...,...,...,...,...,...
56799,62,Cabo Verde,CPV,532.43,530,2018-02-15
57009,61,Cabo Verde,CPV,538.82,532,2018-03-15
57217,58,Cabo Verde,CPV,545.47,539,2018-04-12
57428,58,Cabo Verde,CPV,545.47,545,2018-05-17


In [68]:
# for a row in wc_cleaned, find the home and away team
for index, row in wc_cleaned.iterrows():

    # if the home_team_total_points is zero AND away_team_total_points is non-zero, then fill in the home_team_total_points from team_rankings if team_rankings.country_full contains the home_team
    if row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points != 0:
        home_team = row.home_team
        # print("Home Team missing points = ", home_team)
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[0]

    # if the home_team_total_points is non-zero AND away_team_total_points is zero, then fill in the away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points != 0 and row.away_team_total_fifa_points == 0:
        away_team = row.away_team
        # print("Away Team missing points = ", away_team)
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[0]

    # if both home_team_total_points and away_team_total_points are zero, then fill in both home_team_total_points and away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points == 0:
        home_team = row.home_team
        # print("Home Team (and Away Team) missing points = ", home_team)
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[0]

        away_team = row.away_team
        # print("Away Team (and Home Team) missing points = ", away_team)
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[0]

    # if both home_team_total_points and away_team_total_points are non-zero, then do nothing
    else:
        pass

# check wc_cleaned    
wc_cleaned

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8264,2004-09-03,Spain,Scotland,Europe,Europe,3,67,788,535,1,...,No,Draw,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8
8266,2004-09-04,Austria,England,Europe,Europe,90,7,488,732,2,...,No,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,662,523,3,...,No,Win,77.0,74.0,80.5,78.7,79.0,71.8,75.7,70.2
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,514,595,1,...,No,Lose,78.0,78.0,68.8,77.0,69.2,70.5,79.7,78.5
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,722,610,2,...,No,Win,97.0,79.0,91.8,92.3,87.5,79.2,81.3,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,No,Lose,87.0,89.0,75.2,84.7,75.8,80.8,85.7,85.5
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,No,Draw,75.0,75.0,74.8,78.7,80.0,76.5,72.7,73.8
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,No,Win,76.0,83.0,74.2,77.0,78.0,70.0,72.3,73.5
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,No,Lose,77.0,65.0,73.5,73.7,75.0,76.2,74.7,68.2


In [69]:
# check if there are any zero values in home_team_total_fifa_points and away_team_total_fifa_points
wc_cleaned[(wc_cleaned.home_team_total_fifa_points == 0) | (wc_cleaned.away_team_total_fifa_points == 0)]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
9505,2006-03-01,Tunisia,Serbia,Africa,Europe,23,47,701,0,0,...,No,Lose,67.0,75.0,71.5,70.7,73.0,78.5,82.3,81.5
9596,2006-05-27,Serbia,Uruguay,Europe,South America,47,22,0,689,1,...,No,Draw,75.0,82.0,78.5,82.3,81.5,79.0,83.7,80.0
9660,2006-06-11,Serbia,Netherlands,Europe,Europe,47,3,0,768,0,...,No,Lose,75.0,86.0,78.5,82.3,81.5,84.0,91.0,85.2
9673,2006-06-16,Argentina,Serbia,South America,Europe,9,47,746,0,6,...,No,Win,87.0,75.0,86.5,86.0,87.0,78.5,82.3,81.5
9690,2006-06-21,Côte d'Ivoire,Serbia,Africa,Europe,32,47,669,0,3,...,No,Win,69.0,75.0,81.2,81.7,75.8,78.5,82.3,81.5


In [70]:
# for a row in wc_cleaned, find the home and away team and impute with the 2nd most immediate previous points after the match date
for index, row in wc_cleaned.iterrows():

    # if the home_team_total_points is zero AND away_team_total_points is non-zero, then fill in the home_team_total_points from team_rankings
    if row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points != 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[1]

    # if the home_team_total_points is non-zero AND away_team_total_points is zero, then fill in the away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points != 0 and row.away_team_total_fifa_points == 0:
        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[1]

    # if both home_team_total_points and away_team_total_points are zero, then fill in both home_team_total_points and away_team_total_points from team_rankings
    elif row.home_team_total_fifa_points == 0 and row.away_team_total_fifa_points == 0:
        home_team = row.home_team
        home_team_rankings = team_rankings[team_rankings.country_full == home_team]
        home_team_rankings_afterMatch = home_team_rankings[home_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'home_team_total_fifa_points'] = home_team_rankings_afterMatch.previous_points.iloc[1]

        away_team = row.away_team
        away_team_rankings = team_rankings[team_rankings.country_full == away_team]
        away_team_rankings_afterMatch = away_team_rankings[away_team_rankings.rank_date > row.date]
        wc_cleaned.loc[index, 'away_team_total_fifa_points'] = away_team_rankings_afterMatch.previous_points.iloc[1]

    # if both home_team_total_points and away_team_total_points are non-zero, then do nothing
    else:
        pass

In [71]:
# find match where home_team = Germany, away_team = Costa Rica
wc_cleaned[(wc_cleaned.home_team == 'Germany') & (wc_cleaned.away_team == 'Costa Rica')]['tournament']

9653    FIFA World Cup
Name: tournament, dtype: object

In [72]:
wc_cleaned.tournament.value_counts()

Friendly                                      1730
FIFA World Cup qualification                  1054
UEFA Euro qualification                        452
UEFA Nations League                            236
FIFA World Cup                                 196
UEFA Euro                                      161
Copa América                                   133
African Cup of Nations                          86
Confederations Cup                              52
African Cup of Nations qualification            45
Gold Cup                                        40
African Nations Championship                    33
EAFF Championship                               13
Kirin Challenge Cup                             11
AFC Asian Cup                                    9
Kirin Cup                                        8
Superclásico de las Américas                     8
Nations Cup                                      6
African Nations Championship qualification       6
CONCACAF Nations League        

In [73]:
# check if there are any zero values in home_team_total_fifa_points and away_team_total_fifa_points
wc_cleaned[(wc_cleaned.home_team_total_fifa_points == 0) | (wc_cleaned.away_team_total_fifa_points == 0)]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score


## <u>Add Attendance Figures To 'FIFA World Cup' Matches

In [74]:
# import matches_1930_2018.csv
matches = pd.read_csv('../data/matches_1930_2018.csv')
matches

# rename home_score and away_score and Date to home_team_score and away_team_score and date
matches.rename(columns={'home_score': 'home_team_score', 'away_score': 'away_team_score', 'Date': 'date'}, inplace=True)

In [75]:
matches.columns

Index(['home_team', 'away_team', 'home_team_score', 'home_xg', 'home_penalty',
       'away_team_score', 'away_xg', 'away_penalty', 'home_manager',
       'home_captain', 'away_manager', 'away_captain', 'home_goals',
       'away_goals', 'Attendance', 'Venue', 'Officials', 'date', 'Score',
       'Referee', 'Notes', 'Round', 'Host', 'Year'],
      dtype='object')

In [76]:
# filter wc_cleaned to 'FIFA World Cup' only
wc_matches = wc_cleaned[wc_cleaned.tournament == 'FIFA World Cup']

In [77]:
# left join attendance to wc_matches on 'date','home_team','away_team','home_team_score','away_team_score'
wc_matches = pd.merge(wc_matches, matches[['date','home_team','away_team','home_team_score','away_team_score','Attendance']], on=['date','home_team','away_team','home_team_score','away_team_score'], how='left')


In [79]:
# find where wc_matches Attendance is null
wc_matches[wc_matches.Attendance.isnull()]

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,Attendance
3,2006-06-11,Serbia,Netherlands,Europe,Europe,47,3,724,768,0,...,Lose,75.0,86.0,78.5,82.3,81.5,84.0,91.0,85.2,NaN
4,2006-06-12,USA,Czech Republic,North America,Europe,5,2,756,772,0,...,Lose,84.0,94.0,77.2,76.7,78.5,82.2,83.0,83.5,NaN
9,2006-06-16,Argentina,Serbia,South America,Europe,9,47,746,724,6,...,Win,87.0,75.0,86.5,86.0,87.0,78.5,82.3,81.5,NaN
11,2006-06-17,Italy,USA,Europe,North America,13,5,728,756,1,...,Draw,95.0,84.0,90.2,89.3,86.5,77.2,76.7,78.5,NaN
17,2006-06-21,Côte d'Ivoire,Serbia,Africa,Europe,32,47,669,724,3,...,Win,69.0,75.0,81.2,81.7,75.8,78.5,82.3,81.5,NaN
37,2010-06-12,England,USA,Europe,North America,8,14,1068,957,1,...,Draw,81.0,82.0,85.2,85.0,85.8,73.8,73.7,78.2,NaN
50,2010-06-18,Slovenia,USA,Europe,North America,25,14,860,957,2,...,Draw,78.0,82.0,70.2,72.7,71.8,73.8,73.7,78.2,NaN
59,2010-06-22,South Africa,France,Africa,Europe,83,9,392,1044,2,...,Win,71.0,86.0,72.2,72.0,75.2,84.0,83.3,84.5,NaN
69,2010-06-26,USA,Ghana,North America,Africa,14,32,957,800,1,...,Lose,82.0,66.0,73.8,73.7,78.2,71.8,73.3,80.2,NaN
94,2014-06-16,Ghana,USA,Africa,North America,37,13,704,1034,1,...,Lose,71.0,81.0,70.5,72.7,78.5,73.0,76.0,77.0,NaN


In [80]:
# display wc_matches where attendance is null
wc_matches[wc_matches.Attendance.isnull()]

# manually impute attendance for the 16 matches
wc_matches.loc[wc_matches.index == 3, 'Attendance'] = 43000
wc_matches.loc[wc_matches.index == 4, 'Attendance'] = 52000
wc_matches.loc[wc_matches.index == 9, 'Attendance'] = 52000
wc_matches.loc[wc_matches.index == 11, 'Attendance'] = 46000
wc_matches.loc[wc_matches.index == 17, 'Attendance'] = 66000
wc_matches.loc[wc_matches.index == 37, 'Attendance'] = 38646
wc_matches.loc[wc_matches.index == 50, 'Attendance'] = 45573
wc_matches.loc[wc_matches.index == 59, 'Attendance'] = 39415
wc_matches.loc[wc_matches.index == 69, 'Attendance'] = 34976
wc_matches.loc[wc_matches.index == 94, 'Attendance'] = 39760
wc_matches.loc[wc_matches.index == 110, 'Attendance'] = 40123
wc_matches.loc[wc_matches.index == 112, 'Attendance'] = 69112
wc_matches.loc[wc_matches.index == 124, 'Attendance'] = 41876
wc_matches.loc[wc_matches.index == 132, 'Attendance'] = 51227
wc_matches.loc[wc_matches.index == 167, 'Attendance'] = 41970
wc_matches.loc[wc_matches.index == 182, 'Attendance'] = 78011

In [81]:
# filter to world cup quali matches
wc_quali_matches = wc_cleaned[wc_cleaned.tournament == 'FIFA World Cup qualification']
wc_quali_matches.isna().sum()

date                             0
home_team                        0
away_team                        0
home_team_continent              0
away_team_continent              0
home_team_fifa_rank              0
away_team_fifa_rank              0
home_team_total_fifa_points      0
away_team_total_fifa_points      0
home_team_score                  0
away_team_score                  0
tournament                       0
city                             0
country                          0
neutral_location                 0
shoot_out                        0
home_team_result                 0
home_team_goalkeeper_score       0
away_team_goalkeeper_score       0
home_team_mean_defense_score     0
home_team_mean_offense_score     0
home_team_mean_midfield_score    0
away_team_mean_defense_score     0
away_team_mean_offense_score     0
away_team_mean_midfield_score    0
dtype: int64

In [82]:
wc_cleaned

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
8264,2004-09-03,Spain,Scotland,Europe,Europe,3,67,788,535,1,...,No,Draw,94.0,84.0,86.5,89.3,89.5,80.2,79.7,81.8
8266,2004-09-04,Austria,England,Europe,Europe,90,7,488,732,2,...,No,Draw,83.0,88.0,76.2,73.0,74.0,90.5,88.7,91.2
8271,2004-09-04,Croatia,Hungary,Europe,Europe,25,76,662,523,3,...,No,Win,77.0,74.0,80.5,78.7,79.0,71.8,75.7,70.2
8276,2004-09-04,Iceland,Bulgaria,Europe,Europe,80,41,514,595,1,...,No,Lose,78.0,78.0,68.8,77.0,69.2,70.5,79.7,78.5
8278,2004-09-04,Italy,Norway,Europe,Europe,9,38,722,610,2,...,No,Win,97.0,79.0,91.8,92.3,87.5,79.2,81.3,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,2022-06-14,Poland,Belgium,Europe,Europe,26,2,1544,1827,0,...,No,Lose,87.0,89.0,75.2,84.7,75.8,80.8,85.7,85.5
23911,2022-06-14,Ukraine,Republic of Ireland,Europe,Europe,27,47,1535,1449,1,...,No,Draw,75.0,75.0,74.8,78.7,80.0,76.5,72.7,73.8
23912,2022-06-14,Bosnia and Herzegovina,Finland,Europe,Europe,59,57,1388,1406,3,...,No,Win,76.0,83.0,74.2,77.0,78.0,70.0,72.3,73.5
23913,2022-06-14,Romania,Montenegro,Europe,Europe,48,70,1446,1342,0,...,No,Lose,77.0,65.0,73.5,73.7,75.0,76.2,74.7,68.2


## Join *WC_Matches* Back to *WC_Quali_Matches*